
$\qquad$ $\qquad$$\qquad$  **TDA231/DIT370 Discrete Optimization: Home Assignment 3 -- LP Duality and the Primal-Dual Algorithm** <br />
$\qquad$ $\qquad$$\qquad$                   **During grading time, direct queries regarding Q-1,2 to David & Q-3 to Marc** <br />
$\qquad$ $\qquad$$\qquad$                     **Due Date: 03/03/2025** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Name, Personal No., Email** <br />



---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook and equations, wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook does not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Problem 1.

Consider the following LP problem:

\begin{alignat*}{2}
\max \ &4x_1-2x_2+5x_3+6x_4+7x_5\\
\\
\textrm{s.t} \quad
&2x_1 + 2x_2 - 4x_3 + 4x_4 + 8x_5 &&\leq 6\\
&2x_1 + \ \ {}x_2 - 2x_3 - \ \ x_4 - 3x_5 &&\geq -1\\
&5x_1 - 2x_2 + 4x_3 + 4x_4 + 2x_5 &&= 5\\
&2x_1 - 2x_2 + 5x_3 + 3x_4 + \ \ x_5 &&\leq 4\\
&\hspace{5.3cm} \vec x &&\geq \vec 0
\end{alignat*}

* (4 points) Write the LP dual of this problem.
* (3 points) Use CVXPY to compute the primal and dual optimum solutions and compare their values.
* (3 points) Check the complementary slackness conditions.

In [ ]:
import cvxpy as cp
import numpy as np

# Data
c = np.array([4, -2, 5, 6, 7], dtype=float)

A = np.array([
    [ 2,  2, -4,  4,  8],
    [ 2,  1, -2, -1, -3],
    [ 5, -2,  4,  4,  2],
    [ 2, -2,  5,  3,  1],
], dtype=float)

b = np.array([6, -1, 5, 4], dtype=float)

# Primal (max)
x = cp.Variable(5, nonneg=True)

primal_constraints = [
    A[0] @ x <= b[0],
    A[1] @ x >= b[1],
    A[2] @ x == b[2],
    A[3] @ x <= b[3],
]
primal_prob = cp.Problem(cp.Maximize(c @ x), primal_constraints)
primal_prob.solve(solver=cp.GLPK)   

print("=== PRIMAL ===")
print("status:", primal_prob.status)
print("opt value:", primal_prob.value)
print("x*:", np.array(x.value).round(6))


# Dual (min)

y1 = cp.Variable(nonneg=True)   
y2 = cp.Variable()              
y3 = cp.Variable()              
y4 = cp.Variable(nonneg=True)   

y = cp.hstack([y1, y2, y3, y4])

dual_constraints = [
    y2 <= 0,
    A.T @ y >= c
]
dual_prob = cp.Problem(cp.Minimize(b @ y), dual_constraints)
dual_prob.solve(solver=cp.GLPK)

print("\n=== DUAL ===")
print("status:", dual_prob.status)
print("opt value:", dual_prob.value)
print("y*:", np.array(y.value).round(6))

# Compare values
print("\n=== STRONG DUALITY CHECK ===")
print("primal - dual:", float(primal_prob.value - dual_prob.value))

# Complementary slackness

xv = np.array(x.value).flatten()
yv = np.array(y.value).flatten()

# primal slacks
slack1 = b[0] - A[0] @ xv                
surplus2 = A[1] @ xv - b[1]              
slack4 = b[3] - A[3] @ xv                

# dual slacks
dual_slack = A.T @ yv - c               

print("\n=== COMPLEMENTARY SLACKNESS ===")
print("slack1 (<=):", slack1)
print("surplus2 (>=):", surplus2)
print("slack4 (<=):", slack4)
print("dual slack A^T y - c:", np.round(dual_slack, 8))

print("\nProducts y_i * slack_i (should be ~0):")
print("y1*slack1:", yv[0]*slack1)
print("y2*surplus2:", yv[1]*surplus2)
print("y4*slack4:", yv[3]*slack4)

print("\nProducts x_j * (A^T y - c)_j (should be ~0):")
print(np.round(xv * dual_slack, 10))


=== PRIMAL ===
status: optimal
opt value: 9.220338983050846
x*: [0.627119 2.813559 1.542373 0.       0.661017]

=== DUAL ===
status: optimal
opt value: 9.220338983050848
y*: [ 0.525424 -0.372881  0.338983  1.      ]

=== STRONG DUALITY CHECK ===
primal - dual: -1.7763568394002505e-15

=== COMPLEMENTARY SLACKNESS ===
slack1 (<=): 8.881784197001252e-16
surplus2 (>=): 6.661338147750939e-16
slack4 (<=): 8.881784197001252e-16
dual slack A^T y - c: [ 0.          0.         -0.          0.83050847  0.        ]

Products y_i * slack_i (should be ~0):
y1*slack1: 4.666700171305743e-16
y2*surplus2: -2.4838888008562817e-16
y4*slack4: 8.881784197001252e-16

Products x_j * (A^T y - c)_j (should be ~0):
[ 0.  0. -0.  0.  0.]


Problem 1 — Primal/Dual results, strong duality, and complementary slackness

Primal optimum
The solver returns an optimal primal solution  
$x^\star = (0.627119,\; 2.813559,\; 1.542373,\; 0,\; 0.661017)$  
with objective value  
$c^T x^\star = 9.220338983050846$.  
Status: **optimal**.


Dual optimum
The solver returns an optimal dual solution  
$y^\star = (y_1,y_2,y_3,y_4) = (0.525424,\; -0.372881,\; 0.338983,\; 1)$  
with objective value  
$b^T y^\star = 9.220338983050848$.  
Status: **optimal**.

Strong duality
The objective values match up to numerical tolerance:  
$c^T x^\star - b^T y^\star \approx -1.7763568394\cdot 10^{-15} \approx 0$.  

Therefore strong duality holds (the primal and dual optimal values are equal).


Complementary slackness
Primal constraint slacks (from the solver output):**
- Constraint 1 (a $\le$ constraint): $\text{slack}_1 \approx 8.88\cdot 10^{-16}\approx 0$
- Constraint 2 (a $\ge$ constraint), using surplus $(a_2^T x^\star - b_2)$: $\text{surplus}_2 \approx 6.66\cdot 10^{-16}\approx 0$
- Constraint 4 (a $\le$ constraint): $\text{slack}_4 \approx 8.88\cdot 10^{-16}\approx 0$
(Constraint 3 is an equality and is tight by definition.)

The products are approximately zero:
- $y_1\cdot \text{slack}_1 \approx 0$
- $y_2\cdot \text{surplus}_2 \approx 0$
- $y_4\cdot \text{slack}_4 \approx 0$

Dual slack:
The solver reports  
$A^T y^\star - c = (0,\;0,\;0,\;0.83050847,\;0)$.

Complementary slackness for variables requires  
$x_j^\star\big((A^T y^\star)_j - c_j\big)=0$ for each $j$,  
and the computed products are all (numerically) zero.

In particular, $(A^T y^\star)_4 - c_4 = 0.83050847 > 0$ implies $x_4^\star = 0$, which matches the primal solution.



Conclusion
Both primal and dual solve to optimality, the optimal values agree (strong duality), and complementary slackness holds up to numerical tolerance. This confirms that the reported solutions $x^\star$ and $y^\star$ are consistent and correct.


# Problem 2.

Consdier the LP problem:
\begin{alignat*}{2}
\max \ &6x_1 - 5x_3\\
\\
\textrm{s.t} \quad
&6x_1 - 3x_2 + x_3 &&= 2\\
&3x_1 + 4x_2 + x_3 &&\leq 5\\
&\ \ x_1 - 7x_2 &&\leq 5\\
&\hspace{2.45cm} x_1 &&\geq 0\\
&\hspace{2.45cm} x_2 &&\leq 0\\
&\hspace{2.45cm} x_3 &&\text{ unrestricted}
\end{alignat*}

* (3 points) Write the LP dual of this problem.
* (4 points) Consider the feasible solution $\vec x = (0,0,2)$ to the primal. Check if this solution is optimal by using the complementary slackness conditions to write down the corresponding dual solution.
* (3 points) Use complementary slackness to check if the primal feasible solution $\vec x = (1,0,-4)$ is optimal.

# Problem 3.

Consider the primal-dual algorithm for vertex cover discussed in class.
* (4 points) Run the algorithm by hand on the graph in the figure below (from your previous homework). Show the values of the primal and dual variables at each iteration.
* (6 points) Implement the primal-dual algorithm as a python script to compute (approximate) vertex covers and run it for the random graph $G(100,0.1)$ from the previous homework.

<img src="https://tinyurl.com/tsnuz2c" alt="Drawing" style="width: 180px;"/>

If the image does not load, try the direct link: https://tinyurl.com/tsnuz2c